# TME 2 - 2 : Inférence collective

## Partie 1 - Classifieur local

In [60]:
import pandas as pd
import numpy as np

# Constantes : 
path = 'WebKB/'
univ = 'wisconsin'
percentage_unlabeled = 0.4

# On parse les données
#path = '/Users/ACHANGER/Desktop/UPMC/FDMS-2/WebKB/'
# Fichier 'content' contient des lignes « [URL]\t[Attributs:suite de nombres]\t[Étiquette] »
content = pd.read_csv(path + 'content/'+univ+'.content', sep ='\t', header=None)
# Fichier 'cites' contient des liens « [destination] [source] »
cites   = pd.read_csv(path + 'cites/'+univ+'.cites',     sep =' ', header=None)

# Les identifiants des sites web sont leur URL :
M = len(content)
print("Nombre de sites :", M)
N = content.shape[1] - 2
print("Nombre d'attributs par site :", N)
n_labels = 5
url = content.iloc[:,0] 
labels = content.iloc[:,-1] 
attributes = content.drop([0, N+1],axis=1)

def clean_url(u):
    return u.replace("http://", "").replace("www.", "")

url = url.apply(clean_url)
print("\n=== url dataframe ===")
print(url.head())
print("\n=== labels dataframe ===")
print(labels.head())
print("Distribution :", np.unique(labels, return_counts=True))
print("\n=== attributes dataframe ===")
print(attributes.head())

Nombre de sites : 265
Nombre d'attributs par site : 1703

=== url dataframe ===
0                            robios8.me.wisc.edu
1    robios8.me.wisc.edu/~lumelsky/lumelsky.html
2                           cae.wisc.edu/~ece552
3                                    cs.wisc.edu
4                             cs.wisc.edu/condor
Name: 0, dtype: object

=== labels dataframe ===
0    project
1    faculty
2     course
3     course
4    project
Name: 1704, dtype: object
Distribution : (array(['course', 'faculty', 'project', 'staff', 'student'], dtype=object), array([ 76,  35,  22,  10, 122]))

=== attributes dataframe ===
   1     2     3     4     5     6     7     8     9     10    ...   1694  \
0     0     0     0     0     0     0     0     0     1     0  ...      0   
1     0     0     0     0     0     0     0     0     1     0  ...      0   
2     0     0     0     0     0     0     0     0     0     0  ...      0   
3     0     0     0     0     0     0     0     0     0     0  ...      

In [41]:
from collections import deque
from pprint import pprint

# Conversion des labels depuis string vers int
classes_number = len(np.unique(labels))
url_to_labels = dict(zip(url, labels))
unique_labels = sorted(np.unique(labels))
labels_to_id = dict(zip(unique_labels, range(len(unique_labels))))
url_pos = dict(zip(url, range(M)))

# Parsing du réseau depuis fichier 'cites'
network = {}
for line in cites.iterrows():
    src = clean_url(line[1][1])
    dest = clean_url(line[1][0])
    if src in network:
        network[src].add(dest)
    else:
        network[src] = set([dest])
        
# Suppression de labels
remove_cluster = True
if remove_cluster:
    rand_noeud = np.random.choice(list(network.keys()))
    voisins = deque(network[rand_noeud]) # voisins est de type "queue" : FIFO
    unlabeled = set([rand_noeud])
    while len(unlabeled) < M*percentage_unlabeled and voisins: # tq 'voisin' non vide
        #print("voisins : ", voisins)
        voisin = voisins.popleft()
        unlabeled.add(voisin)
        if voisin in network:
             voisins.extend(set(network[voisin]) - unlabeled)
        if not voisins: # si 'voisin' vide
            voisins = deque([np.random.choice(list(network.keys()))])        
else:
    unlabeled = np.random.choice(url, size=int(M*percentage_unlabeled))
    print("\nUnlabeled items:")
    print(unlabeled)
    example_page = np.random.choice(url)
    print("Page d'exemple:", example_page, "dans unlabeled :", example_page in unlabeled)

print("=== unlabeled ===")
pprint(unlabeled)

example_page = np.random.choice(url)
print("Voisins de "+example_page+" :")
if example_page in network:
    print("(url, label, unlabeled)")
    print([(v, labels_to_id[url_to_labels[v]], v in unlabeled) for v in network[example_page]])
else:
    print("Pas de citations depuis cette page")


=== unlabeled ===
{'cs.wisc.edu',
 'cs.wisc.edu/condor',
 'cs.wisc.edu/condor/next.html',
 'cs.wisc.edu/coral',
 'cs.wisc.edu/exodus',
 'cs.wisc.edu/paradise',
 'cs.wisc.edu/shore',
 'cs.wisc.edu/~agupta/agupta.html',
 'cs.wisc.edu/~alain/alain.html',
 'cs.wisc.edu/~amos/amos.html',
 'cs.wisc.edu/~arch/uwarch/courses/cs354.html',
 'cs.wisc.edu/~arch/uwarch/courses/cs552.html',
 'cs.wisc.edu/~arch/uwarch/courses/cs752.html',
 'cs.wisc.edu/~arch/uwarch/courses/cs757.html',
 'cs.wisc.edu/~ashraf/ashraf.html',
 'cs.wisc.edu/~bach/bach.html',
 'cs.wisc.edu/~bart/cs736.html',
 'cs.wisc.edu/~ben/ben.html',
 'cs.wisc.edu/~bestor/bestor.html',
 'cs.wisc.edu/~bestor/cs302/cs302.html',
 'cs.wisc.edu/~bockrath/bockrath.html',
 'cs.wisc.edu/~brad/brad.html',
 'cs.wisc.edu/~breach/breach.html',
 'cs.wisc.edu/~caol/caol.html',
 'cs.wisc.edu/~carey/carey.html',
 'cs.wisc.edu/~cs110/cs110.html',
 'cs.wisc.edu/~cs132-1/cs132.html',
 'cs.wisc.edu/~cs132-3/cs132.html',
 'cs.wisc.edu/~cs302/course.html',
 

In [42]:
from graphviz import Digraph

graph = Digraph()
labeled = set(url) - unlabeled

for n in labeled:
    graph.node(n,n)
for n in unlabeled:
    graph.node(n, n, {'color':'blue'})
for noeud, voisins in network.items():
    graph.edges([(noeud, voisin) for voisin in voisins])

graph.render("img/graph")

'img/graph.pdf'

In [43]:
# Comptage des classes des voisins
# Dans 'count' : 1 ligne = 1 url, 1 colonne = 1 classe (classes triées par ordre alphabétique) 
count = np.zeros((M, len(unique_labels)))

# On itère sur tous les noeuds qui ont des liens sortants
for noeud, voisins in network.items():
    position = url_pos[noeud]
    for voisin in network[noeud]:
        voisin_label = labels_to_id[url_to_labels[voisin]]
        if voisin not in unlabeled:
            count[position, voisin_label] += 1
            
print("Voisinage de la page d'exemple :", count[url_pos[example_page]])

Voisinage de la page d'exemple : [ 0.  0.  0.  0.  1.]


In [53]:
# Classification

from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier 
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier


def aggreg_neighbors(X, mode="count"):
    """ Transforms the matrix of neighborhood given a mode
    :param X: The matrix of neighbors
    :param mode: string, optional (default="count")
        Specificies the mode of the aggregator (Available: "mode, proportion, count, exist")
    :return: A matrix of the same shape as X
    """    
    if mode == "mode": # garde seulement le plus fréquent
        mat = np.zeros_like(X)
        if len(X.shape) > 1:
            for line, idx in zip(mat, X.argmax(axis=1)):
                line[idx] = 1
        else:
            mat[X.argmax()] = 1
        return mat
    elif mode == "proportion":
        mat = X.copy()
        for line in mat:
            line /= np.linalg.norm(line, ord=1)
        return mat
    elif mode == "count":
        return X.copy()
    elif mode == "exist":
        mat = X.copy()
        mat[mat != 0] = 1
        return mat
    else:
        print("Erreur, mode inconnu :" + mode)
        return None

    
def count_transform(count):
    if len(count.shape) > 1:
        return np.concatenate((aggreg_neighbors(count),
                          aggreg_neighbors(count, mode="mode"),
                          aggreg_neighbors(count, mode="exist")),
                          axis=1)
    else:
        return np.concatenate((aggreg_neighbors(count),
                          aggreg_neighbors(count, mode="mode"),
                          aggreg_neighbors(count, mode="exist")))
        
X = np.concatenate((np.array(attributes), count_transform(count)),
                   axis=1)
print("X.shape :", X.shape)
X_scale = preprocessing.scale(X)
Y = np.array([labels_to_id[lab] for lab in labels])

idx = set(range(M))
test_idx = set([url_pos[u] for u in unlabeled])
train_idx = list(idx - test_idx)
test_idx = list(test_idx)

X_train, X_test = X_scale[train_idx], X_scale[test_idx]
Y_train, Y_test = Y[train_idx], Y[test_idx]
true_labels = np.array([labels_to_id[lab] for lab in labels])[test_idx]


estimators = {"rf":{"clf":RandomForestClassifier(),
                    "params":{"n_estimators":np.arange(2, 100, 10),
                              "max_features":("auto", "sqrt", "log2", None),
                              "max_depth":np.arange(2, 20, 5)}},
              "svc":{"clf":SVC(),
                     "params":{"C":np.logspace(-2, 2, 7),
                              "kernel":("linear", "poly", "rbf"),
                              "class_weight":(None, "balanced")}},
              "knn":{"clf":KNeighborsClassifier(),
                     "params":{"n_neighbors":np.arange(2, 100, 10), 
                               "weights":("uniform", "distance"), 
                               "algorithm":("auto", "ball_tree", "kd_tree", "brute")}},
              "adaboost":{"clf":AdaBoostClassifier(),
                          "params":{"base_estimator":(None, RandomForestClassifier()), 
                                    "n_estimators":np.linspace(5, 50, 5, dtype=int), 
                                    "learning_rate":np.logspace(-1,1,3)}}
             }

choice = "adaboost"

best_clf = None
best_score = 0
estimators_aggreg = []
for choice in estimators.keys():
    print("Searching best "+choice+" estimator")
    gridsearch = GridSearchCV(estimators[choice]["clf"], estimators[choice]["params"], n_jobs=-1, verbose=0)
    %time gridsearch.fit(X_train, Y_train)
    estimators_aggreg.append((choice, gridsearch.best_estimator_))
    print("Score :", gridsearch.best_score_)
    if gridsearch.best_score_ > best_score:
        best_clf = gridsearch.best_estimator_
        best_score = gridsearch.best_score_
    print(20*"=")

print("Aggrégation avec VotingClassifier")
vote_clf = VotingClassifier(estimators_aggreg)
vote_clf.fit(X_train, Y_train)
vote_score = vote_clf.score(X_test, true_labels) 
print("Score vote :", vote_score)

if vote_score > best_score:
    best_clf = vote_clf

print("Estimateur choisi :", type(best_clf))

print("Score sur le test set :", best_clf.score(X_test, true_labels))
print("Proportion des labels :", dict(np.unique(Y, return_counts=True)))
print("Prédiction sur le test set :", best_clf.predict(X_test))
print("Vrais labels :", true_labels)



X.shape : (265, 1718)
Searching best adaboost estimator
CPU times: user 244 ms, sys: 28 ms, total: 272 ms
Wall time: 4.16 s
Score : 0.8427672955974843
Searching best knn estimator
CPU times: user 300 ms, sys: 20 ms, total: 320 ms
Wall time: 1.87 s
Score : 0.6037735849056604
Searching best rf estimator
CPU times: user 1.28 s, sys: 20 ms, total: 1.3 s
Wall time: 17.5 s
Score : 0.8553459119496856
Searching best svc estimator
CPU times: user 436 ms, sys: 16 ms, total: 452 ms
Wall time: 2.81 s
Score : 0.7861635220125787
Aggrégation avec VotingClassifier
Score vote : 0.660377358491
Estimateur choisi : <class 'sklearn.ensemble.forest.RandomForestClassifier'>
Score sur le test set : 0.669811320755


ValueError: dictionary update sequence element #0 has length 5; 2 is required

## Partie 2 : ICA

On va réutiliser les données de la partie précédente pour la 1ère phase, _bootstraping_

In [56]:
from sklearn.metrics import accuracy_score


test_idx = sorted(test_idx)
ordering = np.copy(test_idx)

print("Unlabeled indexes : ", test_idx)
#print("Y before :", Y)
Y[test_idx] = best_clf.predict(X_test)
#print("Y after  :", Y)
old_labels = np.zeros_like(Y[test_idx]) - 1
print("Y[test_idx] :", Y[test_idx])
print("true_labels :", true_labels)

max_iter = 5
i = 0
while not np.allclose(old_labels, Y[test_idx]) and i<max_iter:
    print("======= Itération %d =======" % i)
    # On itère sur tous les noeuds qui ont des liens sortants
    np.random.shuffle(ordering)
    for position in ordering:
        noeud = url[position]
        count_noeud = np.zeros((classes_number,))
        if noeud in network:
            for voisin in network[noeud]:
                vlabel = Y[url_pos[voisin]]
                count_noeud[vlabel] += 1
                transform = count_transform(count_noeud)
                X[position, N:] = transform
    old_labels = Y[test_idx]
    Y[test_idx] = best_clf.predict(X[test_idx])
    print("Y[test_idx] :", Y[test_idx])
    print("true_labels :", true_labels)
    
    i += 1
    
print("Fini.")
print("Score : ", accuracy_score(Y[test_idx], true_labels))

Unlabeled indexes :  [0, 1, 3, 4, 5, 6, 8, 9, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 26, 27, 28, 30, 33, 35, 36, 42, 43, 46, 47, 49, 52, 53, 63, 65, 66, 69, 71, 75, 76, 81, 84, 85, 86, 92, 98, 101, 102, 104, 107, 111, 112, 117, 118, 119, 120, 123, 126, 128, 129, 131, 132, 134, 139, 140, 141, 142, 143, 145, 146, 147, 160, 161, 164, 167, 168, 177, 184, 188, 190, 191, 192, 195, 196, 201, 202, 209, 211, 212, 213, 214, 215, 221, 224, 229, 233, 234, 236, 242, 243, 248, 249, 254, 255, 256, 261, 263]
Y[test_idx] : [4 4 4 4 4 4 4 4 2 4 4 1 4 4 4 4 0 1 0 4 4 0 0 4 4 4 1 4 0 0 4 0 0 4 0 4 4
 1 4 4 0 0 1 4 0 4 0 4 0 0 4 0 4 4 4 4 4 0 4 4 0 4 1 4 4 4 4 0 0 1 4 4 4 4
 4 4 2 4 1 4 4 0 4 0 4 4 2 4 2 4 0 4 1 4 0 4 4 4 4 4 4 4 4 1 0 4]
true_labels : [2 1 0 2 2 2 2 2 2 4 4 1 0 0 0 0 4 1 0 4 4 0 4 4 4 4 1 0 0 0 0 0 0 0 0 0 0
 1 4 2 0 0 1 4 0 2 4 1 4 0 4 0 4 4 4 4 4 0 4 4 4 4 1 4 3 1 4 0 0 1 4 4 2 4
 4 2 2 1 1 4 4 0 4 0 1 4 2 2 2 4 0 4 1 4 0 4 4 4 4 4 4 3 2 1 0 1]
======= Itération 0 =======
Y[test_idx] :

## Partie 3 : Propagation de labels

* $f_\theta(x_i)$ est le score du site n°$i$, de représentation $x_i$ pour le label représenté par $\theta$

In [ ]:
from scipy.optimize import minimize

class LabelPropag():
    
    def __init__(self, graph_reg=1, l2_reg=1):
        self.graph_reg = graph_reg
        self.l2_reg = l2_reg
        
    def f(self, theta, x):
        """ Return the score for x and label theta"""
        return theta.dot(x)

    def objective(self, theta, X, train_idx, Y, graph):
       # print("Appel d'objective avec theta=", theta)
        s = 0
        for i in train_idx:
            s += self.hinge_loss(self.f(theta, X[i]), Y[i])
            
        #print("X.shape :",X.shape)
        for i, neighbors in graph.items():
            for j in neighbors:
                #print("i,j :", i,j)
                s += self.graph_reg * (self.f(theta, X[i]) - self.f(theta, X[j]))**2
        s += self.l2_reg * np.linalg.norm(theta)**2
        #print("Valeur : ",s)
        return s
        
    def fit(self, X, train_idx, labels, graph, n_labels):
        """
        :params graph: a dictionary of neighborhood
        :params n_labels: number of labels """
        self.thetas = np.zeros((n_labels, X.shape[1]))
        # Init Y
        Y = np.zeros((X.shape[0], n_labels))
        Y[:] = -1
        for i,label in enumerate(labels):
            Y[i, label] = 1
        
        # Trouver meilleurs theta_l
        for label in range(n_labels):
            print("Recherche de theta_%d..." % label)
            self.thetas[label] = minimize(lambda t:self.objective(t, X, train_idx, Y[:,label], graph), 
                                          self.thetas[label],
                                          method="TNC",
                                          tol=1e-2,
                                          options={"maxiter":1000, "disp":True}).x
            print("→ ", self.thetas[label])
    
        
    def hinge_loss(self, score1, score2):
        if score1*score2 > 0:
            return 0
        else:
            return 1
        
    def predict(self, X):
        return np.array([np.argmax([self.f(theta, x) 
                        for theta in self.thetas]) for x in X])
    
    def score(self, X, Y):
        return accuracy_score(self.predict(X), Y)
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            self.parameter = value
        return self

X = np.array(attributes)
Y = np.array([labels_to_id[label] for label in labels])

#X_train, X_test = X[train_idx], X[test_idx]
#Y_train, Y_test = Y[train_idx], Y[test_idx]

label_propag = LabelPropag(graph_reg=1, l2_reg=1)
graph = {url_pos[noeud]:[url_pos[voisin] for voisin in network[noeud]] for noeud in network.keys()}
label_propag.fit(X, train_idx, Y, graph, n_labels=n_labels)
print(label_propag.score(X_test, Y_test))

Recherche de theta_0...
→  [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   1.17968566e-10
   7.47134254e-10   0.00000000e+00]
Recherche de theta_1...


0.490566037736
